In [ ]:
import os, re, pandas as pd

# base directory
base_dir = r"E:\Eye_Tracking_Education_Video\experiment2_analysis"
events_path = os.path.join(base_dir, "Event_Experiment2_try10.xlsx") # event file
all_fix_dir = os.path.join(base_dir, "all_fixations")

out_cond1 = os.path.join(base_dir, "Fixations_condition1.csv")
out_cond2 = os.path.join(base_dir, "Fixations_condition2.csv")

def numeric_subject_id_from_name(name: str) -> int:
    m = re.search(r"sub-(\d+)", name, re.IGNORECASE)
    return int(m.group(1)) if m else -1

def stim_from_name(name: str) -> int:
    m = re.search(r"task-stim(\d+)", name, re.IGNORECASE)
    return int(m.group(1)) if m else -1

def condition_from_name(name: str) -> int:
    low = name.lower()
    if "ses-01" in low: return 1
    if "ses-02" in low: return 2
    return -1

# read events
events = pd.read_excel(events_path)
events.columns = [c.strip().lower() for c in events.columns]
events["video"] = events["video"].astype(int)

rows_c1, rows_c2 = [], []

for f in os.listdir(all_fix_dir):
    if not f.endswith("_desc-fixations.tsv"):
        continue
    tsv = os.path.join(all_fix_dir, f)

    subj_num = numeric_subject_id_from_name(f)
    stim = stim_from_name(f)
    cond = condition_from_name(f)
    if subj_num < 0 or stim < 0 or cond < 0:
        continue

    # subject ID for ses-02 +30 # 记得改
    subj_num_cond = subj_num + 30 if cond == 2 else subj_num

    evs = events[events["video"] == stim]
    if evs.empty:
        continue

    df = pd.read_csv(tsv, sep="\t")
    st_col = "start_time" if "start_time" in df.columns else "onset"
    x_col = "x_pos" if "x_pos" in df.columns else "x"
    y_col = "y_pos" if "y_pos" in df.columns else "y"

    # 用 start_time 判断属于哪个 event
    for _, ev in evs.iterrows():
        s, e, trial = float(ev["start"]), float(ev["end"]), int(ev["event"])
        mask = (df[st_col] >= s) & (df[st_col] < e)
        sel = df.loc[mask, [x_col, y_col]]
        target = rows_c1 if cond == 1 else rows_c2 if cond == 2 else None
        if target is None:
            continue
        for _, r in sel.iterrows():
            target.append({
                "SubjectID": subj_num_cond,
                "TrialID": trial,
                "FixX": r[x_col],
                "FixY": r[y_col],
            })

pd.DataFrame(rows_c1, columns=["SubjectID","TrialID","FixX","FixY"])\
  .to_csv(out_cond1, index=False, encoding="utf-8-sig")
pd.DataFrame(rows_c2, columns=["SubjectID","TrialID","FixX","FixY"])\
  .to_csv(out_cond2, index=False, encoding="utf-8-sig")


✅ 已保存: E:\Eye_Tracking_Education_Video\experiment2_analysis\Fixations_condition1.csv
✅ 已保存: E:\Eye_Tracking_Education_Video\experiment2_analysis\Fixations_condition2.csv


In [4]:
import pandas as pd

file_path = "Experiment2_Fix_S_try10.xlsx"
df = pd.read_excel(file_path)

if "StimuliID" in df.columns:
    df["StimuliID"] = df["StimuliID"].astype(str) + ".jpg"

output_path = "Experiment2_Fix_S_try10.xlsx"
df.to_excel(output_path, index=False)

output_path


'Experiment2_Fix_S_try10.xlsx'